<a href="https://colab.research.google.com/github/frankzamma/AntiPiracyPlatform/blob/main/Notebook/PPO_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training e Test con PPO
In questo notebook è presente il training di Q*Bert sfruttando l'algoritmo PPO

## Download Repository

In [1]:
from google.colab import userdata

In [2]:
!git clone https://{userdata.get('TokenGithub')}"@github.com/amigli/Q-Bert_RL.git"

Cloning into 'Q-Bert_RL'...
remote: Enumerating objects: 606, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 606 (delta 46), reused 27 (delta 17), pack-reused 528 (from 1)
Receiving objects: 100% (606/606), 10.49 MiB | 7.81 MiB/s, done.
Resolving deltas: 100% (383/383), done.


In [3]:
%cd Q-Bert_RL/

/content/Q-Bert_RL


## Installazione dei requirements

In [4]:
!pip install gymnasium

In [5]:
!pip install ale-py

In [6]:
!pip install moviepy

In [7]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [8]:
!pip install wandb

## Algoritmo

In [9]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import ale_py
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from stable_baselines3.common.logger import configure
import torch
from torch.utils.tensorboard import SummaryWriter
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from EnvironmentWrappers.RewardFunction import RewardFunction
from EnvironmentWrappers.ObsRewardWrapper import ObsRewardWrapper
import wandb
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.utils import get_linear_fn
import torch as th
from wandb.integration.sb3 import WandbCallback
from stable_baselines3.common.atari_wrappers import NoopResetEnv


## Wandb

In [10]:
!wandb login --relogin

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [11]:
config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 7_000_000,
    "env_name": "ALE/Qbert-ram-v5",
}
run = wandb.init(
    project="QBERT-RL",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
    entity="Q-BertRLTeam"
)


/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: frank581-fgz (frankzamma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


## Training

In [12]:
gym.register_envs(ale_py)

def make_env(env_id):
    def _init():
        env = gym.make(env_id)
        env = ObsRewardWrapper(env)
        env = NoopResetEnv(env, noop_max=30)
        env = Monitor(env)
        # env = OurRewardWrapper(env)
        return env
    return _init

In [13]:
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[256, 512, 256, 128, 64], vi=[256, 512, 256, 128, 64]))


In [14]:
from typing import Callable


def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [15]:
lr_schedule = linear_schedule(0.0001)

num_envs = 25
envs = DummyVecEnv([make_env("ALE/Qbert-ram-v5") for _ in range(num_envs)])

model = PPO(
    "MlpPolicy",
    envs,
    verbose=1,
    n_steps = 1024,
    batch_size=64,
    ent_coef= 0.03,
    learning_rate=lr_schedule,
    policy_kwargs=policy_kwargs,
    tensorboard_log=f"runs/{run.id}"
    )

Using cuda device


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [16]:
env = gym.make("ALE/Qbert-ram-v5")
env = ObsRewardWrapper(env)

eval_callback = EvalCallback(env, best_model_save_path="./BestModels/",
                             log_path="./logs/", eval_freq=250,
                             deterministic=True, render=False)

wandb_callback = WandbCallback(
        gradient_save_freq=500,
        model_save_path=f"models/{run.id}",
        verbose=2,
    )

model.learn(total_timesteps = 7_000_000, callback=[wandb_callback,eval_callback])

wandb.finish()

# Valutazione del modello
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Ricompensa media: {mean_reward:.2f}, deviazione standard: {std_reward:.2f}")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Logging to runs/vx73kc3x/PPO_1


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Output streaming troncato alle ultime 5000 righe.
|    explained_variance   | 0.84        |
|    learning_rate        | 5.83e-05    |
|    loss                 | 0.21        |
|    n_updates            | 1140        |
|    policy_gradient_loss | -0.0149     |
|    value_loss           | 1.5         |
-----------------------------------------
Eval num_timesteps=2925000, episode_reward=8.80 +/- 8.35
Episode length: 812.80 +/- 191.76
---------------------------------
| eval/              |          |
|    mean_ep_length  | 813      |
|    mean_reward     | 8.8      |
| time/              |          |
|    total_timesteps | 2925000  |
---------------------------------
Eval num_timesteps=2931250, episode_reward=12.20 +/- 8.03
Episode length: 917.80 +/- 148.03
---------------------------------
| eval/              |          |
|    mean_ep_length  | 918      |
|    mean_reward     | 12.2     |
| time/              |          |
|    total_timesteps | 2931250  |
-------------------------------

KeyboardInterrupt: 

In [17]:
run.finish()

eval/mean_ep_length,▁▁███▅▅▅▅▅▄▅▃▅▆▆▅▃▅▅▄▅▅▇█▅▅▅▅▅██▇▇▆█▅▇▇▅
eval/mean_reward,▃▁▄▁▂▃▅▄▅▅▆▇▇█▅▄▅▄▅▅▇▅▅▄▅▅█▄▆▇▆▇▆█▆▅▆▅▇▅
global_step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇████
rollout/ep_len_mean,▂▁▁▁▁▃▃▃▄▄▄▅▅▅▆▅▅▅▆▆▆▆▇▇▇▇▆▆▆▇▇▇▆▆▇▇▆▇██
rollout/ep_rew_mean,▂▁▂▂▂▁▁▂▂▃▄▄▅▅▆▆▆▆▅▆▆▆▆▇▇▆▆▇▇▆▇▇▇▇▇▇▇▇▇█
time/fps,█▇▇▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train/approx_kl,▃▂▂▁▂▆▅▅██▇█▆▇▆▆▇▇▇█▇▇██▆▆█▆▆▇▆▆▇▇▆▆▅▆▆▅
train/clip_fraction,▅▂▁▃▃▄▆▆▅▄▆▆▅▄▆▅██▅▇▆▇▅▅▆▇▇▅▅▇▇▅▅▆▅▅▄▃▄▄
train/clip_range,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/entropy_loss,▂▁▁▁▁▂▂▂▂▂▂▂▂▃▃▄▄▄▄▄▄▄▄▅▄▄▅▅▅▅▅▅▅▆▅▇▆▇▇█
train/explained_variance,▁▅▆▇▆▆▆▆▅▇▆▇▇▇▇██▇██▇▇▇▇▇██▇▇█▇█▇██▇▇▇██


In [18]:
env = gym.make("ALE/Qbert-ram-v5", render_mode="rgb_array")
env = ObsRewardWrapper(env)

In [19]:
# DummyVecEnv per compatibilità con Stable-Baselines3
env = DummyVecEnv([lambda: env])

In [20]:
# Registra video
video_folder = "./videos/"
env = VecVideoRecorder(
    env,               # Ambiente
    video_folder,      # Cartella per salvare i video
    record_video_trigger=lambda x: x % 10000000 == 0,  # Registra ogni 1000 passi
    video_length=10000000 # Durata massima del video in passi
)

In [21]:
# Resetta l'ambiente per registrare un episodio
obs = env.reset()

# Registra 3 episodi
for episode in range(3):
    obs = env.reset()
    for _ in range(10000000):  # Durata massima dell'episodio
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, dones, info = env.step(action)
        if dones[0]:  # L'episodio è terminato
            break

env.close()  # Salva il video

/usr/local/lib/python3.11/dist-packages/moviepy/config_defaults.py:1: DeprecationWarning: invalid escape sequence '\P'
  """


Moviepy - Building video /content/Q-Bert_RL/videos/rl-video-step-0-to-step-10000000.mp4.
Moviepy - Writing video /content/Q-Bert_RL/videos/rl-video-step-0-to-step-10000000.mp4



Moviepy - Done !
Moviepy - video ready /content/Q-Bert_RL/videos/rl-video-step-0-to-step-10000000.mp4


Moviepy - Building video /content/Q-Bert_RL/videos/rl-video-step-0-to-step-10000000.mp4.
Moviepy - Writing video /content/Q-Bert_RL/videos/rl-video-step-0-to-step-10000000.mp4



Moviepy - Done !
Moviepy - video ready /content/Q-Bert_RL/videos/rl-video-step-0-to-step-10000000.mp4
